In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import json
from sklearn.model_selection import train_test_split

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
datasets = ['All_Beauty_5', 'AMAZON_FASHION_5', 'Arts_Crafts_and_Sewing_5', 'Clothing_Shoes_and_Jewelry_5', 'Luxury_Beauty_5']

for i in datasets:
    input_file = f'D:/aml_data/{i}.json'
    output_file = f'D:/aml_data/{i}.csv'

    with open(input_file, "r", encoding="utf-8") as input_json, open(output_file, "w", encoding="utf-8") as output_csv:
        csv_writer = csv.writer(output_csv)
        flag = 0
        for line in input_json:
                dic = json.loads(line)
                if flag == 0:
                    csv_writer.writerow(dic.keys())
                    flag = 1
                csv_writer.writerow(dic.values())

print("Done")

Done


In [4]:
df_1 = pd.read_csv('D:/aml_data//All_Beauty_5.csv', usecols=['overall', 'reviewText', "summary"])
df_2 = pd.read_csv('D:/aml_data//AMAZON_FASHION_5.csv', usecols=['overall', 'reviewText', "summary"])
df_3 = pd.read_csv('D:/aml_data//Arts_Crafts_and_Sewing_5.csv', usecols=['overall', 'reviewText', "summary"])
df_4 = pd.read_csv('D:/aml_data//Clothing_Shoes_and_Jewelry_5.csv', usecols=['overall', 'reviewText', "summary"])
df_test = pd.read_csv('D:/aml_data//Luxury_Beauty_5.csv', usecols=['overall', 'reviewText', "summary"])

In [5]:
df_train = pd.concat([df_1, df_2, df_3, df_4], axis=0, ignore_index=True)
df_train = df_train.dropna()
df_test = df_test.dropna()

df_train = df_train[df_train["overall"] != '3']
df_train["label"] = df_train["overall"].apply(lambda rating : 1 if str(rating) > '3' else 0)

df_test = df_test[df_test["overall"] != '3']
df_test["label"] = df_test["overall"].apply(lambda rating : 1 if str(rating) > '3' else 0)

In [6]:
print(f'First review = {df_train.loc[0, "reviewText"]}')
print(f'First review has length = {len(df_train.loc[0, "reviewText"])}\n ')
print(f'First review overall rating = {df_train.loc[0, "overall"]}')
print(f'First review binary rating = {df_train.loc[0, "label"]}')

First review = As advertised. Reasonably priced
First review has length = 32
 
First review overall rating = 5.0
First review binary rating = 1


In [7]:
print("TensorFlow Version:", tf.__version__)

# Check available GPUs
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

TensorFlow Version: 2.9.1
Num GPUs Available:  1
Name: /physical_device:GPU:0   Type: GPU


In [8]:
X = pd.DataFrame(df_train, columns = ["reviewText"])
y = pd.DataFrame(df_train, columns = ["label"])

train_x, val_x, train_y, val_y = train_test_split(X, y, train_size=0.8, random_state=50, shuffle=True)

In [9]:
df_Xtrain = pd.DataFrame(train_x,columns=['reviewText'])
df_ytrain = pd.DataFrame(train_y,columns=['label'])
df_train = pd.concat([df_ytrain, df_Xtrain], axis=1)
df_Xval = pd.DataFrame(val_x,columns=['reviewText'])

In [10]:
print(len(df_train))
print(len(df_Xval))

print(df_train.head())
print(df_Xval.head())

8111964
2027992
         label                reviewText
2567756      1           Nice sunglasses
6585046      1            Fits perfectly
7522833      0  You get what you pay for
1465266      1              Soft leather
6909483      1                Five Stars
                       reviewText
7259421  Durable and Easy to Move
8237271      I Love these sandals
6135424                Five Stars
4290872                Five Stars
9102221      A Very Good T- shirt


**Train:**

In [11]:
max_tokens = 1000
output_sequence_length = 100
pad_to_max_tokens = True

df_train['reviewText'] = df_train['reviewText'].fillna('').astype(str)

encoder = tf.keras.layers.TextVectorization(max_tokens=max_tokens, output_sequence_length=output_sequence_length, pad_to_max_tokens=pad_to_max_tokens)

text_ds = tf.data.Dataset.from_tensor_slices(df_train['reviewText']).batch(128)
encoder.adapt(text_ds)
vocab = np.array(encoder.get_vocabulary())

train_ds = tf.data.Dataset.from_tensor_slices((df_train['reviewText'], df_train['label'])).batch(128)
train_ds = train_ds.map(lambda x, y: (encoder(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

**Test:**

In [12]:
df_Xval['reviewText'] = df_Xval['reviewText'].fillna('').astype(str)

text_test_ds = tf.data.Dataset.from_tensor_slices(df_Xval['reviewText']).batch(128)
df_val = text_test_ds.map(lambda x: encoder(x))

AUTOTUNE = tf.data.AUTOTUNE
test_ds = df_val.cache().prefetch(buffer_size=AUTOTUNE)

In [13]:
print(test_ds)

<PrefetchDataset element_spec=TensorSpec(shape=(None, 100), dtype=tf.int64, name=None)>


In [14]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(5):
        print("Review:", text_batch.numpy()[i])
        print("Label:", label_batch.numpy()[i])
        print("---")

Review: [ 34 691   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
Label: 1
---
Review: [ 79 192   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
Label: 1
---
Review: [ 37  77  76  37 538  10   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

In [15]:
embedding_dimension = 128
embedding_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(vocab),
                              output_dim=embedding_dimension,
                              input_length=100,
                              name="embedding"),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

embedding_model.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

embedding_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          128000    
                                                                 
 bidirectional (Bidirectiona  (None, 100, 256)         263168    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                82176     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                        

In [16]:
embedding_weights = embedding_model.get_layer('embedding').get_weights()[0]
print(f'Dimension of the embedding vector: \n{embedding_weights.shape}')

Dimension of the embedding vector: 
(1000, 128)


In [17]:
print(vocab[500:550])

['lots' 'expect' 'quickly' 'huge' 'us' 'issue' 'customer' 'between' 'show'
 'gave' 'flat' 'although' 'says' 'job' 'decent' 'either' 'house' 'couldnt'
 'travel' 'instead' 'head' 'seller' 'carry' 'needs' 'fall' 'each' 'washed'
 'weather' 'stuff' 'lbs' 'outside' 'pink' 'hours' 'clothes' 'sized'
 'paper' 'ankle' 'decided' 'pay' 'havent' 'somewhat' 'machine' 'thank'
 'someone' 'necklace' 'bright' 'gold' 'bras' 'everyday' 'stretchy']


In [18]:
def get_word_index(word, encoder):
    try:
        return encoder.get_vocabulary().index(word)
    except ValueError:
        return None

words = ['dad', 'today']

for word in words:
    word_index = get_word_index(word, encoder)
    if word_index is not None:
        word_vector = embedding_weights[word_index]
        print(f'{word}: {np.round(word_vector, 3)}')
    else:
        print(f'"{word}" not in vocabulary')


"dad" not in vocabulary
today: [ 0.044 -0.046  0.031 -0.023  0.037  0.017 -0.003  0.021 -0.023 -0.028
 -0.033  0.041  0.044  0.027 -0.005 -0.01  -0.008  0.042  0.047  0.043
  0.027 -0.048  0.047 -0.016 -0.023  0.035  0.002 -0.011  0.048  0.024
 -0.019  0.042 -0.02   0.038  0.048 -0.008 -0.029 -0.01  -0.031  0.018
  0.033 -0.018  0.02   0.023  0.041  0.044 -0.038  0.048 -0.     0.026
 -0.012 -0.028 -0.023  0.007 -0.008 -0.036 -0.027 -0.036  0.014 -0.017
  0.045 -0.035  0.038 -0.049 -0.038 -0.021 -0.043  0.012  0.03   0.017
  0.029  0.023  0.035 -0.022  0.031 -0.021 -0.022 -0.039  0.002  0.008
  0.035  0.025 -0.048 -0.021 -0.004  0.013  0.014  0.005 -0.031 -0.049
  0.046 -0.013  0.033  0.001 -0.002 -0.039  0.032  0.031 -0.032  0.046
  0.042  0.031  0.02  -0.034  0.022 -0.023 -0.001 -0.026 -0.002  0.042
  0.006 -0.043 -0.05   0.039  0.012 -0.037 -0.023  0.008  0.027  0.04
 -0.043 -0.009  0.019 -0.005 -0.014  0.022  0.001  0.034]


In [19]:
embedding_model.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

embedding_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          128000    
                                                                 
 bidirectional (Bidirectiona  (None, 100, 256)         263168    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                82176     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                        

In [20]:
val_x['reviewText'] = val_x['reviewText'].fillna('').astype(str)
val_ds = tf.data.Dataset.from_tensor_slices((val_x['reviewText'], val_y)).batch(128)
val_ds = val_ds.map(lambda x, y: (encoder(x), y))
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [21]:
hist_ex1 = embedding_model.fit(train_ds, validation_data=val_ds, epochs=10, verbose=1)

Epoch 1/10
63375/63375 [==============================] - 2872s 45ms/step - loss: 0.1496 - accuracy: 0.9428 - val_loss: 0.1274 - val_accuracy: 0.9483
Epoch 2/10
63375/63375 [==============================] - 2112s 33ms/step - loss: 0.1258 - accuracy: 0.9489 - val_loss: 0.1245 - val_accuracy: 0.9494
Epoch 3/10
63375/63375 [==============================] - 2061s 33ms/step - loss: 0.1235 - accuracy: 0.9498 - val_loss: 0.1236 - val_accuracy: 0.9498
Epoch 4/10
63375/63375 [==============================] - 2267s 36ms/step - loss: 0.1224 - accuracy: 0.9503 - val_loss: 0.1232 - val_accuracy: 0.9500
Epoch 5/10
63375/63375 [==============================] - 2279s 36ms/step - loss: 0.1218 - accuracy: 0.9506 - val_loss: 0.1230 - val_accuracy: 0.9502
Epoch 6/10
63375/63375 [==============================] - 2432s 38ms/step - loss: 0.1214 - accuracy: 0.9507 - val_loss: 0.1228 - val_accuracy: 0.9501
Epoch 7/10
63375/63375 [==============================] - 2422s 38ms/step - loss: 0.1214 - accuracy:

In [26]:
predictions = embedding_model.predict(val_ds)

15844/15844 [==============================] - 151s 9ms/step


array([[0.9992512 ],
       [0.9986981 ],
       [0.9998454 ],
       ...,
       [0.99839824],
       [0.26230383],
       [0.71835023]], dtype=float32)

In [28]:
# For classification models
predicted_classes = (predictions > 0.5).astype(int)


# Extract actual labels from the validation dataset
actual_labels = np.concatenate([y for x, y in val_ds], axis=0)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(actual_labels, predicted_classes)
print("Accuracy on Validation Set:", accuracy)



Accuracy on Validation Set: 0.9499608479717869


In [23]:
embedding_model.save('initial_model')

INFO:tensorflow:Assets written to: initial_model\assets


INFO:tensorflow:Assets written to: initial_model\assets


In [29]:
tf.saved_model.save(embedding_model, 'initial_model_full')

INFO:tensorflow:Assets written to: initial_model_full\assets


INFO:tensorflow:Assets written to: initial_model_full\assets
